In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random 
import time
from scipy.stats import rankdata

In [2]:
def TDT(n):
    b = n[0] + n[2] + 2*n[3]
    c = n[1] + n[2] + 2*n[4]
    
    if b == 0 and c == 0:
        return 0
    elif b == 0:
        return 1000000000
    else:
        return (b-c)**2/(b+c)

In [3]:
def lap_normal(stats, s, epsilon, K, m):
    g = sorted(stats, reverse=True)
    pri_data = g + np.random.laplace(loc = 0.0, scale = (2*K*s)/epsilon, size=np.size(stats))
    pri_r = np.argsort(pri_data)
    
    return pri_r[m-K:m]

In [4]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_stats(stats, s, epsilon, K, m):
    g = sorted(stats, reverse=True)
    
    h = np.zeros(m)
    for i in range(m):
        h[i] = g[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * h[i] / (2*K*s))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [5]:
def SHD(cc, n):
    T = TDT(n)
    
    d = np.zeros(2)
    N = np.zeros(6)
    
    for k in range(6):
        N[k] = n[k]
    
    if T < cc:
        while T < cc:
            if N[4] > 0:
                N[4] -= 1
            elif N[1] > 0:
                N[1] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[0] -= 1
            N[3] += 1
            
            T = TDT(N)
            d[0] -= 1
        
        T = TDT(n)
        for k in range(6):
            N[k] = n[k]
        
        while T < cc:
            if N[3] > 0:
                N[3] -= 1
            elif N[0] > 0:
                N[0] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[1] -= 1
            N[4] += 1
            
            T = TDT(N)
            d[1] -= 1
        
        if d[0] > d[1]:
            return d[0]
        else:
            return d[1]
    
    else:
        step = 0
        
        if n[0] + 2*n[3] > n[1] + 2*n[4]:
            while T >= cc:
                if n[3] > 0:
                    n[3] -= 1
                elif n[0] > 0:
                    n[0] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[1] -= 1
                n[4] += 1
            
                T = TDT(n)
                step += 1
        else:
            while T >= cc:
                if n[4] > 0:
                    n[4] -= 1
                elif n[1] > 0:
                    n[1] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[0] -= 1
                n[3] += 1
            
                T = TDT(n)
                step += 1
        
        return step-1

def exp_shd(n, epsilon, K, m):
    shd = np.zeros(m)
    cc = 19.5
    
    for i in range(m):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [6]:
def FPA(stats, sensitivity, epsilon, K, m):
    rank = np.argsort(stats)
    g = np.zeros(m)
    for i in range(m):
        g[i] = stats[i]
        
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    pri_x = np.zeros(m)
    
    s = int(m/(2*K))
    
    l = 2*K*sensitivity*np.sqrt(s)/epsilon
    
    prime_y[0:s] = y[0:s] + np.random.laplace(loc= 0.0, scale = l, size = s)
    
    pri_x = np.fft.ifft(prime_y).real
    
    pri_r = np.argsort(pri_x)
    
    #for i in range(K):
    #    for j in range(m):
    #        if pri_r[m-i-1] == rank[m-j-1]:
    #            pri_r[m-i-1] = j
    #            break
    
    return pri_r[m-K:m]

In [7]:
def extended_FPA(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(M)-i] = prime_y.real[i]
            prime_y.imag[int(M)-i] = -prime_y.imag[i]
    
    #prime_g = np.fft.ifft(prime_y)
    
    l = 2*K*sensitivity/epsilon
    
    pri_x = np.zeros(m)
    
    noise = np.zeros(m)
    
    for j in range(m):
        p = np.random.laplace(loc= 0.0, scale = l, size = 1)
        prime_y.real[0] += p
        for k in range(1,s):
            prime_y.real[k] += p * math.cos(-2*j*k*math.pi/m)
            prime_y.imag[k] += p * math.sin(-2*j*k*math.pi/m)
            prime_y.real[m-k] = prime_y.real[k]
            prime_y.imag[m-k] = -prime_y.imag[k]
            
        pri_x[j] = np.fft.ifft(prime_y).real[j]
        
    pri_r = np.argsort(pri_x)
    
    return pri_r[m-K:m]

In [8]:
def lap_DFT(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(M)-i] = prime_y.real[i]
            prime_y.imag[int(M)-i] = -prime_y.imag[i]
    
    prime_g = np.fft.ifft(prime_y)
    
    pri_g = prime_g.real + np.random.laplace(loc = 0.0, scale = (2*K)*((2*s-1)/m)*sensitivity/epsilon, size=m)
    
    pri_r = np.argsort(pri_g)
    
    return pri_r[m-K:m]

In [9]:
def exp_DFT(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(m)-i] = prime_y.real[i]
            prime_y.imag[int(m)-i] = -prime_y.imag[i]
    
    prime_g = np.fft.ifft(prime_y).real
    
    h = np.zeros(m)
    for i in range(m):
        h[i] = prime_g[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp((epsilon * h[i] / (2*K*sensitivity)) * (m / (2*s-1)))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [10]:
def runTime(K, N, M):
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    sig = random.sample(range(M),10)
    
    for i in range(10):
        S[sig[i]] = random.randint(0,2*N)
        b[sig[i]] = np.random.binomial(S[sig[i]],0.75)
        c[sig[i]] = S[sig[i]] - b[sig[i]]
        n[sig[i]][0] = b[sig[i]]
        n[sig[i]][1] = c[sig[i]]
        n[sig[i]][5] = 2*N - b[sig[i]] - c[sig[i]]
    
    for i in range(M):
        if S[i] == 0:
            S[i] = random.randint(0,2*N)
            b[i] = np.random.binomial(S[i],0.5)
            c[i] = S[i] - b[i]
            n[i][0] = b[i]
            n[i][1] = c[i]
            n[i][5] = 2*N - b[i] - c[i]
        
    stats = np.zeros(M)
    order = np.zeros(M)
    
    for i in range(M):
        order[i] = i
        if b[i] == 0 and c[i] == 0:
            stats[i] = 0
        else:
            stats[i] = (b[i]-c[i])**2/(b[i]+c[i])
    
    x = np.argsort(stats)
    #for i in range(10):
    #    print(x[M-1-i])
    
    nn = np.zeros((M,6))
    for i in range(M):
        nn[i] = n[x[M-1-i]]
    
    sensitivity = 8*(N-1)/N
        
    RT = np.zeros(7)
    
    start = time.time()
    lap_normal(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[0] = end-start
    
    start = time.time()
    exp_stats(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[1] = end-start
    
    start = time.time()
    exp_shd(nn, 3, K, M)
    end = time.time()
    RT[2] = end-start
    
    start = time.time()
    FPA(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[3] = end-start
    
    start = time.time()
    extended_FPA(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[4] = end-start
    
    start = time.time()
    lap_DFT(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[5] = end-start
    
    start = time.time()
    exp_DFT(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[6] = end-start
    
    return RT

In [11]:
#K = 1

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(1,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.0028600692749023438 [sec]
exp_stats_time = 0.009164571762084961 [sec]
exp_shd_time = 0.8547382354736328 [sec]
FPA_time = 0.0018796920776367188 [sec]
extended_FPA_time = 69.17945694923401 [sec]
lap_DFT_time = 0.006672859191894531 [sec]
exp_DFT_time = 0.01491093635559082 [sec]
------------------
lap_normal_time = 0.0028617382049560547 [sec]
exp_stats_time = 0.008391141891479492 [sec]
exp_shd_time = 0.8727719783782959 [sec]
FPA_time = 0.0016109943389892578 [sec]
extended_FPA_time = 68.27783989906311 [sec]
lap_DFT_time = 0.005835771560668945 [sec]
exp_DFT_time = 0.012123346328735352 [sec]
------------------
lap_normal_time = 0.0021278858184814453 [sec]
exp_stats_time = 0.008015155792236328 [sec]
exp_shd_time = 0.8337087631225586 [sec]
FPA_time = 0.0016739368438720703 [sec]
extended_FPA_time = 68.6367678642273 [sec]
lap_DFT_time = 0.007091045379638672 [sec]
exp_DFT_time = 0.013648033142089844 [sec]
------------------
lap_normal_time = 0.002283811569213867 [sec]
exp_stats

In [12]:
#K = 3

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(3,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.0028738975524902344 [sec]
exp_stats_time = 0.019735097885131836 [sec]
exp_shd_time = 0.8421850204467773 [sec]
FPA_time = 0.0015742778778076172 [sec]
extended_FPA_time = 23.191601037979126 [sec]
lap_DFT_time = 0.0032749176025390625 [sec]
exp_DFT_time = 0.02306389808654785 [sec]
------------------
lap_normal_time = 0.0021839141845703125 [sec]
exp_stats_time = 0.017893314361572266 [sec]
exp_shd_time = 0.8279368877410889 [sec]
FPA_time = 0.00160980224609375 [sec]
extended_FPA_time = 23.103806972503662 [sec]
lap_DFT_time = 0.003328084945678711 [sec]
exp_DFT_time = 0.02238011360168457 [sec]
------------------
lap_normal_time = 0.0023119449615478516 [sec]
exp_stats_time = 0.018243789672851562 [sec]
exp_shd_time = 0.8449001312255859 [sec]
FPA_time = 0.0016791820526123047 [sec]
extended_FPA_time = 23.124356031417847 [sec]
lap_DFT_time = 0.003348112106323242 [sec]
exp_DFT_time = 0.02287888526916504 [sec]
------------------
lap_normal_time = 0.002260923385620117 [sec]
exp_stat

In [13]:
#K = 5

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(5,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.0021822452545166016 [sec]
exp_stats_time = 0.030441999435424805 [sec]
exp_shd_time = 0.8573188781738281 [sec]
FPA_time = 0.0016582012176513672 [sec]
extended_FPA_time = 14.102443933486938 [sec]
lap_DFT_time = 0.002891063690185547 [sec]
exp_DFT_time = 0.03472709655761719 [sec]
------------------
lap_normal_time = 0.0023691654205322266 [sec]
exp_stats_time = 0.028593778610229492 [sec]
exp_shd_time = 0.8462100028991699 [sec]
FPA_time = 0.0015521049499511719 [sec]
extended_FPA_time = 14.255841970443726 [sec]
lap_DFT_time = 0.002965688705444336 [sec]
exp_DFT_time = 0.03542137145996094 [sec]
------------------
lap_normal_time = 0.002296924591064453 [sec]
exp_stats_time = 0.02944803237915039 [sec]
exp_shd_time = 0.8531801700592041 [sec]
FPA_time = 0.0014350414276123047 [sec]
extended_FPA_time = 14.316690921783447 [sec]
lap_DFT_time = 0.002971172332763672 [sec]
exp_DFT_time = 0.03501176834106445 [sec]
------------------
lap_normal_time = 0.0024390220642089844 [sec]
exp_stat

In [14]:
#K = 10

N = 150
M = 5000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(10,150,5000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 0.0025510787963867188 [sec]
exp_stats_time = 0.05801200866699219 [sec]
exp_shd_time = 0.861638069152832 [sec]
FPA_time = 0.0014700889587402344 [sec]
extended_FPA_time = 7.099768877029419 [sec]
lap_DFT_time = 0.0026493072509765625 [sec]
exp_DFT_time = 0.06719374656677246 [sec]
------------------
lap_normal_time = 0.0023970603942871094 [sec]
exp_stats_time = 0.058493852615356445 [sec]
exp_shd_time = 0.8679349422454834 [sec]
FPA_time = 0.0014147758483886719 [sec]
extended_FPA_time = 7.315482139587402 [sec]
lap_DFT_time = 0.0027539730072021484 [sec]
exp_DFT_time = 0.06981897354125977 [sec]
------------------
lap_normal_time = 0.0028989315032958984 [sec]
exp_stats_time = 0.06772804260253906 [sec]
exp_shd_time = 0.9310061931610107 [sec]
FPA_time = 0.002006053924560547 [sec]
extended_FPA_time = 8.13555097579956 [sec]
lap_DFT_time = 0.003258228302001953 [sec]
exp_DFT_time = 0.06928396224975586 [sec]
------------------
lap_normal_time = 0.0021767616271972656 [sec]
exp_stats_ti